# Sign Language to Text

## Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import cv2
import joblib

## Importing Dataset

### Preprocessing the Training Set

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set_bw',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 color_mode='grayscale',
                                                 class_mode = 'categorical')

Found 8640 images belonging to 9 classes.


### Preprocessing the Test Set

In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set_bw',
                                           target_size = (128,128),
                                           batch_size = 32,
                                            color_mode='grayscale',
                                           class_mode = 'categorical')

Found 2160 images belonging to 9 classes.


## Buildding the CNN Model

### Initializing the CNN

In [4]:
cnn = tf.keras.models.Sequential()

### Adding first convolution layer

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation="relu",
                               input_shape=[128, 128, 1]))

### Adding first max pooling layer

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding second convolution layer

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=5,
                               activation="relu"))

### Adding second max pooling layer

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Flattening the matrix of features

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Adding the fully connected layer

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))
cnn.add(tf.keras.layers.Dropout(0.40))
cnn.add(tf.keras.layers.Dense(units=96, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.40))
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

### Adding the output layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=9, activation="softmax"))

## Training the CNN

In [12]:
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [13]:
cnn.fit(training_set, validation_data = test_set, epochs=3)

Epoch 1/3
270/270 [==============================] - 195s 721ms/step - loss: 0.7078 - accuracy: 0.7419 - val_loss: 8.8799e-04 - val_accuracy: 1.0000
Epoch 2/3
270/270 [==============================] - 121s 449ms/step - loss: 0.0633 - accuracy: 0.9782 - val_loss: 1.4448e-04 - val_accuracy: 1.0000
Epoch 3/3
270/270 [==============================] - 132s 489ms/step - loss: 0.0354 - accuracy: 0.9880 - val_loss: 1.1098e-05 - val_accuracy: 1.0000


## Testing the model on Test Set

In [14]:
file = "D:\Mini_Project\dataset\single_prediction/sample8.jpg"
frame = cv2.imread(file)
img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(img, (5, 5), 2)
th3 = cv2.adaptiveThreshold(
    blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
)
ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imwrite(file, res)

True

In [15]:
img = image.load_img("dataset/single_prediction/sample8.jpg", target_size = (128, 128), color_mode='grayscale')
test_image = image.img_to_array(img)
test_image = np.expand_dims(test_image, axis=0)

In [16]:
result = cnn.predict(test_image)
lis = [i for i in range(9)]
prediction={}
prediction['blank'] = result[0][0]
inde = 0
sum = 0
for i in lis:
    prediction[i] = result[0][inde]
    inde += 1
max = 0
output = 0
for key, value in prediction.items():
    if key == "blank":
        continue
    sum += value
    if value > max:
        max = value
        output = key
print(output+1)
# print(prediction)
# print(result)
# print(sum)

8


In [17]:
model_json = cnn.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
cnn.save_weights('model-bw.h5')
print('Weights saved')

Model Saved
Weights saved
